In [46]:
from bs4 import BeautifulSoup
import requests
import time
import re
import glob
import os
import codecs
import mysql.connector
from pandas import DataFrame
import pandas as pd
from progressbar import ProgressBar
pbar = ProgressBar()
from tqdm import tqdm
from selenium import webdriver

In [48]:
headers = {'User-Agent': 'Mozilla/5.0'}
folder = os.getcwd() + '\\reviewmeta_pages\\'
if not os.path.exists(folder):
    os.makedirs(folder)

# read the file we just stored
source_product = pd.read_csv('source_product.csv', sep=',')
name = input("enter your first name initial in caps")
if name == "T":
    source_product = source_product[0:200]
if name == "N":
    source_product = source_product[201:400]
if name == "I":
    source_product = source_product[401:600]
if name == "J":
    source_product = source_product[601:800]
if name == "Y":
    source_product = source_product[801:1000]
if name == "D":
    source_product = source_product[1001:1200]
if name == "K":
    source_product = source_product[1201:len(source_product_test)-1]
    
problematic = []

enter your first name initial in capsT


In [16]:
name

'T'

In [17]:
source_product.head()

,Source,product,Profile Ids,RSC
0,amazon,B07LB1L3J9,NaN,0
1,amazon,B07TDCWDRF,NaN,0
2,amazon,B07MXRSZCX,NaN,0
3,amazon,B07CNZWD9G,NaN,0
4,amazon,B07SQZ3VFZ,NaN,0


## Functions 

Given a link this script will fetch data from reviewmeta

In [18]:
def checksource(url):
    if (".fr" in url):
        return "amazon-fr"
    if (".ca" in url):
        return "amazon-ca"
    if (".de" in url):
        return "amazon-de"
    if (".uk" in url):
        return "amazon-uk"
    else:
        return "amazon"

### webscraping from reviewmeta

In [19]:
def data_per_product(cat,number_of_rev,soup,product):
    output = {}
    trust, rev_rate, rev_title, rev_det, rev_link_rm,rev_txt_am,rev_lnk_am = ([] for i in range(7))
    
    if(cat == "good"):
        trust_soup = soup.find_all("div",class_  = "col-sm-2 col-xs-12 text-center review-pass-badge")
        if(len(trust_soup)<number_of_rev):
            temp = soup.find_all("div",class_ = "col-sm-2 col-xs-12 text-center review-warn-badge")
            trust_soup = trust_soup + temp

    elif(cat == "bad"):
        trust_soup = soup.find_all("div",class_  = "col-sm-2 col-xs-12 text-center review-fail-badge")
        if(len(trust_soup)<number_of_rev):
            temp = soup.find_all("div",class_ = "col-sm-2 col-xs-12 text-center review-warn-badge")
            trust_soup = trust_soup + temp

    
    for i in range(0,number_of_rev):
        
        trust.append      (int(re.findall(r'[0-9]+',str(trust_soup[i].text))[0])/100)                  #convert trust to 0 to 1 scale
        rev_rate.append   (re.findall(r'[0-5]',str(soup.find_all("div",class_  = "col-md-1 col-xs-2 bw-rating")[i].text))[0])
        rev_title.append  (soup.find_all("div",class_  = "col-md-11 col-xs-10 show-title")[i].text)
        rev_det.append    (soup.find_all("div",class_  = "col-sm-10 col-xs-12")[i].text)
        rev_link_rm.append(soup.find_all("label",class_= "written-by")[i].find("a").get('href'))
        rev_txt_am.append (soup.find_all('div',class_  = "show-actual-review") [i].text)
        rev_lnk_am.append (soup.find_all('div',class_  = "show-actual-review") [i].find("a").get('href'))

    output["product"]          = product
    output["trust"]            = trust   
    output["review_rating"]    = rev_rate
    output["review_title"]     = rev_title
    output["reviewer_details"] = rev_det
    output["reviewer_link_RM"] = rev_link_rm
    output["rvwr_text_Amazon"] = rev_txt_am
    output["rvwr_link_Amazon"] = rev_lnk_am 
    return(output)

### reviwer profile detials clean up

In [40]:
def rev_details_dict(review_temp):
    rev_dict = {"Verified_Purchases": 0, "Nvr_verified_reviewer"   : 0, "Contains_rep_phrases": 0,
                  "high_vol_day_rev": 0, "Critical_Rev_rating"     : 0, "Take_backs"          : 0, 
                 "Take_backs_rating": 0, "Overrep_part"            : 0, "Overrep_wrd_cnt"     : 0,
                 "Easy_grade_rating": 0, "Overlapping_rev_history" : 0, "Brand_Rep_freq"      : 0,
                 "Brand_rep_rating" : 0, "One_hit"                 : 0, "incentivized"        : 0,
                 "Brand_repeater"   : 0, "Brand_Loyalist"          : 0, "Brand_Monogamist"    : 0}

    for rev_metric in review_temp:

        if("Unverified Purchaser" in rev_metric):
            rev_dict["Verified_Purchases"] = 0

        if("Verified Purchaser" in rev_metric):
            rev_dict["Verified_Purchases"] = 1

    #Ex: Never-Verified Reviewer
        if("Never-Verified Reviewer" in rev_metric):
            rev_dict["Nvr_verified_reviewer"]= 1

        if("Contains repetitive phrases" in rev_metric):
            rev_dict["Contains_rep_phrases"] = 1

        if("Created on a high volume day" in rev_metric):
            rev_dict["high_vol_day_rev"]     = 1

        if("Critical Reviewer" in rev_metric):
            rev_dict["Critical_Rev_rating"] = float(rev_metric[-4:-1])

        if("Take-Back Reviewer" in rev_metric):
            rev_dict["Take_backs"]          = int(re.findall(r'[0-9]+',rev_metric)[0])
            rev_dict["Take_backs_rating"]   = float(rev_metric[-4:-1])

    #EX: Overrepresented Participation (posted 2 reviews)        
        if("Overrepresented Participation" in rev_metric):
            rev_dict["Overrep_part"]       = int(re.findall(r'[0-9]+',rev_metric)[0])   

    #EX: Overrepresented word count (3 words)       
        if("Overrepresented word count" in rev_metric):
            rev_dict["Overrep_wrd_cnt"]    = int(re.findall(r'[0-9]+',rev_metric)[0])   

    #Ex: Easy Grader (avg. rating: 4.8)        
        if("Easy Grader" in rev_metric):
            rev_dict["Easy_grade_rating"] = float(rev_metric[-4:-1])

    #Ex: Overlapping Review History: reviewed 1 of 9 top products        
        if("Overlapping Review History" in rev_metric):
            rev_dict["Overlapping_rev_history"] = int(re.findall(r'[0-9]+',rev_metric)[0])/int(re.findall(r'[0-9]+',rev_metric)[1])

    #Brand Repeater (2 of 53 reviews for this brand; avg rating: 5.0)
        if("Brand Repeater" in rev_metric):
            rev_dict["Brand_repeater"]     = 1
            rev_dict["Brand_Rep_freq"]     = (re.findall(r'[0-9]+',rev_metric)[0])+"/"+(re.findall(r'[0-9]+',rev_metric)[1])
            rev_dict["Brand_rep_rating"]   = float(rev_metric[-4:-1])
        
        if("Brand Loyalist" in rev_metric):
            rev_dict["Brand_Loyalist"]     = 1 
            
        if("Brand Loyalist" in rev_metric):
            rev_dict["Brand_Monogamist"]   = 1 

        if("One-Hit Wonder" in rev_metric):
             rev_dict["One_hit"]     = 1 
                
        if ("Admitted to receiving an incentive" in rev_metric):
            rev_dict["incentivized"] = 1
            
    return rev_dict


### give review dataframe for a single prduct 

In [21]:
def get_review_df_per_soup(soup, productname):
    soup_good     = soup.find("div", {"id": "good-reviews"})
    no_of_good    = len(soup_good.find_all("div",id = "sample_reviews"))  
    soup_bad      = soup.find("div", {"id": "bad-reviews"})
    no_of_bad     = len(soup_bad.find_all("div",id = "sample_reviews"))

    good_reviews_sum = pd.DataFrame.from_dict(data_per_product("good",no_of_good,soup_good,productname))
    bad_reviews_sum  = pd.DataFrame.from_dict(data_per_product("bad",no_of_bad,soup_bad,productname))

    review_total              = good_reviews_sum.append(bad_reviews_sum).reset_index(drop=True)
    review_total["Amazon_ID"] = review_total["rvwr_link_Amazon"].apply(lambda x: x[(x.rfind("/"))+1:]) 
    
   # print(review_total)
    
    # split individual review deets into attributes
    all_review_dict=[]
    for i in range(no_of_good + no_of_bad):
        review_temp = review_total["reviewer_details"][i].split("\n")
        rev_details = rev_details_dict(review_temp)
        all_review_dict.append(rev_details)

    #merge all the dictonaries with same key
    review_attributes = {}
    for k in all_review_dict[0]:
        review_attributes[k] = [d[k] for d in all_review_dict]

    review_total= review_total.join(pd.DataFrame.from_dict(review_attributes))
    return review_total

## Downloading review meta pages with time lag on local

In [41]:
for i in tqdm(range(0,len(source_product))):
    time.sleep(1) #Hold 1 seconds before the next scrape.
    num=str(i)
  #  url = source_product["URL"][i]
    product = source_product["product"][i]
    source = checksource (source_product["Source"][i])
    path = os.getcwd() + '\\reviewmeta_pages\\'
    filename = product +"_"+ source + '.htm'
    review_meta_url = 'https://reviewmeta.com/'+ source +'/'+ product
    print(review_meta_url)
    response = requests.get(review_meta_url, headers = headers)
    time.sleep(3)
    with open(path + filename, 'w', encoding="utf8") as file:
        file.write(response.text)

print('Completed')



  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

https://reviewmeta.com/amazon/B07LB1L3J9




  0%|▍                                                                                 | 1/200 [00:04<15:28,  4.67s/it]

https://reviewmeta.com/amazon/B07TDCWDRF




  1%|▊                                                                                 | 2/200 [00:09<15:29,  4.69s/it]

...

https://reviewmeta.com/amazon-fr/B0756GQXKT




100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [15:41<00:04,  4.81s/it]

https://reviewmeta.com/amazon-fr/B00EAPCCW0




100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [15:46<00:00,  4.73s/it]


Completed


In [53]:
review_dataframe_mega = pd.DataFrame()
files = os.listdir(path)
for file_name in files:
   
    f = open("{}{}".format(path,file_name),"r", encoding="utf-8").read()  #creating the path 
    soup = BeautifulSoup(f)
    if (soup.find_all("div",id = "sample_reviews") != []):
        review_dataframe_mega = pd.concat([review_dataframe_mega, get_review_df_per_soup(soup, file_name.split("_")[-2])], ignore_index=True)
    else:
        problematic.append(file_name.split("_")[0])

In [55]:
problematic

['B0018CG40O',
 'B001NI1S0O',
 'B005BOMS00',
 'B00B17ETIU',
 'B00CM8SCV2',
 'B00CWR8628',
 'B00DLQXJ6W',
 'B00IAOB50C',
 'B00K1J7DGE',
 'B00L4CB9X0',
 'B00L4CBA8Y',
 'B00QTCUV0C',
 'B012F869RM',
 'B01HO8U5NC',
 'B0018CG40O',
 'B001NI1S0O',
 'B005BOMS00',
 'B00B17ETIU',
 'B00CM8SCV2',
 'B00CWR8628',
 'B00DLQXJ6W',
 'B00IAOB50C',
 'B00K1J7DGE',
 'B00L4CB9X0',
 'B00L4CBA8Y',
 'B00QTCUV0C',
 'B012F869RM',
 'B01HO8U5NC']

In [56]:
review_dataframe_mega.to_csv("review_dataframe_mega.csv")

In [57]:
review_dataframe_mega

,product,trust,review_rating,review_title,reviewer_details,reviewer_link_RM,rvwr_text_Amazon,rvwr_link_Amazon,Amazon_ID,Verified_Purchases,...,Overrep_wrd_cnt,Easy_grade_rating,Overlapping_rev_history,Brand_Rep_freq,Brand_rep_rating,One_hit,incentivized,Brand_repeater,Brand_Loyalist,Brand_Monogamist
0,B00023N7TG,1.00,5,installed very nicely and quick,\n Verified PurchaserReviewer: Laurie Lalonde\...,https://reviewmeta.com/profile/amazon-ca/A2OH6...,\n\t\t\t\tI purchased this pet door for my scr...,https://www.amazon.ca/gp/customer-reviews/R1NU...,R1NUJUAU9AGLL5,1,...,0,5.0,0.0,0,0.0,0,0,0,0,0
1,B00023N7TG,1.00,4,Doggy Independence!,\n Verified PurchaserReviewer: alcar\n\n Criti...,https://reviewmeta.com/profile/amazon-ca/A16FB...,"\n\t\t\t\tBought this for our small ""min pin"" ...",https://www.amazon.ca/gp/customer-reviews/R2BC...,R2BCHBGRSCZM3Y,1,...,0,0.0,0.0,0,0.0,0,0,0,0,0
2,B00023N7TG,1.00,5,Very pleased with the purchase,\n Verified PurchaserReviewer: Xinavane Mawu K...,https://reviewmeta.com/profile/amazon-ca/AE5KV...,\n\t\t\t\tMy cat learned to use it the same da...,https://www.amazon.ca/gp/customer-reviews/R1FR...,R1FRJXL59II5IA,1,...,0,4.8,0.0,0,0.0,0,0,0,0,0
3,B00023N7TG,1.00,3,Flimsy and not ideal for some pets,\n Verified PurchaserReviewer: Andrea L.\n\n E...,https://reviewmeta.com/profile/amazon-ca/AEXBG...,\n\t\t\t\tI wish I liked this but the problem ...,https://www.amazon.ca/gp/customer-reviews/R276...,R2763N5LVHQ78L,1,...,0,5.0,0.0,0,0.0,0,0,0,0,0
4,B00023N7TG,1.00,4,Must be screwed together,\n Verified PurchaserReviewer: GregH\n\n Take-...,https://reviewmeta.com/profile/amazon-ca/A1OOZ...,\n\t\t\t\tI have purchased 2 of these and inst...,https://www.amazon.ca/gp/customer-reviews/R169...,R169RU505Q79CN,1,...,0,4.7,0.0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,B07XBKYWQ2,0.19,1,Not good,\n Verified PurchaserReviewer: christopher b v...,https://reviewmeta.com/profile/amazon/AHT5F3JP...,\n\t\t\t\tIt will not hold a charge. Now it is...,https://smile.amazon.com/gp/customer-reviews/R...,R5NZA03U327XB,1,...,0,0.0,0.0,0,0.0,1,0,0,0,0
3119,B07XBKYWQ2,0.19,1,Did not work consistently.,\n Verified PurchaserReviewer: Karen\n\n One-H...,https://reviewmeta.com/profile/amazon/AFMBQ5NW...,\n\t\t\t\tDisappointed in this product. It wor...,https://smile.amazon.com/gp/customer-reviews/R...,R1BAUMS94BKB60,1,...,0,0.0,0.0,0,0.0,1,0,0,0,0
3120,B07XBKYWQ2,0.19,5,Great product,\n Verified PurchaserReviewer: Christie Johnso...,https://reviewmeta.com/profile/amazon/AGLQHS77...,\n\t\t\t\tProduct works great [Go to full revi...,https://smile.amazon.com/gp/customer-reviews/R...,R2YKF7JWRM254M,1,...,0,5.0,0.0,0,0.0,1,0,0,0,0
3121,B07XBKYWQ2,0.19,5,Happy,\n Verified PurchaserReviewer: Edwin D Harris\...,https://reviewmeta.com/profile/amazon/AFGXU56U...,\n\t\t\t\tGood product [Go to full review]\n,https://smile.amazon.com/gp/customer-reviews/R...,R2EPWJ76F9YXVS,1,...,0,5.0,0.0,0,0.0,1,0,0,0,0


#for whatever reason the reveiw id is not unique lenght wise : pls see if this can be done nicer

Apologies for the multiple ifs, pls see if these can be done more efficiently 

### Approach 

1) Make this a general function that takes product url as inout and give the review dataframe as output 


2) scrape reviewer_link_RM and get that data